In [1]:
import numpy as np
from scipy.linalg import toeplitz
import matplotlib.pyplot as plt
import cv2

def matrix_to_vector(input):
    input_h, input_w = input.shape
    output_vector = np.zeros(input_h*input_w, dtype=input.dtype)
    # flip the input matrix because last row should go first
    input = np.flipud(input) 
    for i,row in enumerate(input):
        st = i*input_w
        nd = st + input_w
        output_vector[st:nd] = row   
    return output_vector


def vector_to_matrix(input, output_shape):
    output_h, output_w = output_shape
    output = np.zeros(output_shape, dtype=input.dtype)
    for i in range(output_h):
        st = i*output_w
        nd = st + output_w
        output[i, :] = input[st:nd]
    # flip the output matrix up-down to get correct result
    output=np.flipud(output)
    return output


def convolution_as_maultiplication(I, F, print_ir=False):
    I_row_num, I_col_num = I.shape 
    F_row_num, F_col_num = F.shape
    output_row_num = I_row_num + F_row_num - 1
    output_col_num = I_col_num + F_col_num - 1
    if print_ir: print('output dimension:', output_row_num, output_col_num)
    F_zero_padded = np.pad(F, ((output_row_num - F_row_num, 0),
                               (0, output_col_num - F_col_num)),
                            'constant', constant_values=0)
    if print_ir: print('F_zero_padded: ', F_zero_padded)
    toeplitz_list = []
    for i in range(F_zero_padded.shape[0]-1, -1, -1):
        c = F_zero_padded[i, :]  
        r = np.r_[c[0], np.zeros(I_col_num-1)] 
        toeplitz_m = toeplitz(c,r) 
        toeplitz_list.append(toeplitz_m)
        if print_ir: print('F '+ str(i)+'\n', toeplitz_m)
    c = range(1, F_zero_padded.shape[0]+1)
    r = np.r_[c[0], np.zeros(I_row_num-1, dtype=int)]
    doubly_indices = toeplitz(c, r)
    if print_ir: print('doubly indices \n', doubly_indices)
    toeplitz_shape = toeplitz_list[0].shape
    h = toeplitz_shape[0]*doubly_indices.shape[0]
    w = toeplitz_shape[1]*doubly_indices.shape[1]
    doubly_blocked_shape = [h, w]
    doubly_blocked = np.zeros(doubly_blocked_shape)
    b_h, b_w = toeplitz_shape 
    for i in range(doubly_indices.shape[0]):
        for j in range(doubly_indices.shape[1]):
            start_i = i * b_h
            start_j = j * b_w
            end_i = start_i + b_h
            end_j = start_j + b_w
            doubly_blocked[start_i: end_i, start_j:end_j] = toeplitz_list[doubly_indices[i,j]-1]

    if print_ir: print('doubly_blocked: ', doubly_blocked)
    vectorized_I = matrix_to_vector(I)
    if print_ir: print('vectorized_I: ', vectorized_I)
    result_vector = np.matmul(doubly_blocked, vectorized_I)
    if print_ir: print('result_vector: ', result_vector)
    out_shape = [output_row_num, output_col_num]
    output = vector_to_matrix(result_vector, out_shape)
    if print_ir: print('Result of implemented method: \n', output)
    
    return output

if __name__ == "__main__":

#     input signal
    I = np.array([[1,2,3],[4,5,6],[7,8,9]])
    print('I: ', I.shape)

     # filter 
    F = np.array([[1,2,1],[0,0,0],[-1,-2,-1]])
    print('F: ',F.shape)
    print(F)
    result = convolution_as_maultiplication(I, F, print_ir=False)
    print(result)

I:  (3, 3)
F:  (3, 3)
[[ 1  2  1]
 [ 0  0  0]
 [-1 -2 -1]]
[[  1.   4.   8.   8.   3.]
 [  4.  13.  20.  17.   6.]
 [  6.  18.  24.  18.   6.]
 [ -4. -13. -20. -17.  -6.]
 [ -7. -22. -32. -26.  -9.]]
